In [0]:
#@title Example form fields
#@markdown Forms support many types of fields.

square_feet = 2100 #@param {type:"slider", min:0, max:10000, step:100}
bedrooms = 3 #@param {type:"slider", min:0, max:8, step:1}
age_of_dwelling = 30 #@param {type:"number"}
full_bathrooms = 1 #@param {type:"slider", min:0, max:6, step:1}
land_size_sq_ft = 5000 #@param {type:"number"}
#@markdown ---
condo = 0 #@param {type:"slider", min:0, max:1, step:1}
row_house = 0 #@param {type:"slider", min:0, max:1, step:1}
single_family = 1 #@param {type:"slider", min:0, max:1, step:1}
neighborhood = "SLOANS LAKE" #@param ["ACADEMY", "STAPLETON", "SLOANS LAKE", "UNIVERSITY PARK", "HIGHLAND"]


In [0]:
PROJECT_ID = "denver-housing-project" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [0]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS '<path-to-your-service-account-key.json>'

In [0]:
BUCKET_NAME = "du-data-viz-bucket" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}

In [0]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://du-data-viz-bucket/...
ServiceException: 409 Bucket du-data-viz-bucket already exists.


In [0]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://du-data-viz-bucket/model/


In [0]:
print(BUCKET_NAME)

du-data-viz-bucket


In [0]:
! pip install numpy>=1.16.0 scikit-learn==0.20.2

In [0]:
%%writefile predictor.py
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib

class DenverHousingPredictor(object):
  def __init__(self, model_by_neighborhood):
    self._model_by_neighborhood = model_by_neighborhood

  def predict(self, instances, **kwargs):
    
    # Args:
    #     instances: A list of prediction input instances.
    
    #     
    inputs = np.array(instances)
    #     inputs = pd.DataFrame(instances, columns=['AREA_ABG','BED_RMS','AGE','FULL_B','LAND_SQFT','CONDO','ROW_HOUSE','SINGLE_FAMILY'])
    neighborhood =  kwargs['neighborhood']
    
    outputs = self._model_by_neighborhood[neighborhood].predict(inputs)
    
    # array should have this width
    # outputs = self._model.predict(preprocessed_inputs)
    return outputs.tolist()

  @classmethod
  def from_path(cls, model_dir):
    model_by_neighborhood = {}
    for neighborhood in [
     'ARAPAHOE SQUARE',
      'ATHMAR PARK',
      'AURARIA',
      'BALLPARK',
      'BARNUM',
      'BEAR VALLEY',
      'BELMONT HTS',
      'BERKELEY',
      'BREAKERS',
      'CAPITOL HILL',
      'CBD',
      'CHAFFEE PARK',
      'CHERMONT',
      'CHERRY CREEK',
      'CITY PARK WEST',
      'CLAYTON',
      'COLE',
      'COLFAX',
      'COMMONS',
      'COUNTRY CLUB',
      'CREEKVW_SUNSET',
      'CRESTMOOR',
      'CURTIS PARK',
      'DENVER TECH',
      'FIVE POINTS',
      'FORT LOGAN',
      'GLENBROOK',
      'GLOBEVILLE',
      'GOLDSMITH',
      'GREEN VALLEY',
      'HALE',
      'HAMPDEN',
      'HARVEY PARK',
      'HIGHLAND',
      'HILLTOP',
      'HOLLY HILLS',
      'HOLLY RIDGE',
      'HUTCHINSON HILLS',
      'INSPIRATION PT',
      'JEFFERSON PARK',
      'LODO',
      'LOWRY',
      'MAR LEE',
      'MARSTON',
      'MONTBELLO',
      'NORTHFIELD',
      'OVERLAND',
      'PARK FOREST',
      'PARK HILL',
      'PARK VISTA',
      'PARKFIELD',
      'PINE RIDGE',
      'PINE VALLEY',
      'PINECREST VILLAGE',
      'PINEHURST',
      'PLATTE PARK',
      'RANGE VIEW',
      'REGIS',
      'RIVER NORTH',
      'RUBY HILL _ GODSMAN PARK',
      'SEVEN SPRINGS',
      'SKYLAND',
      'SLOANS LAKE',
      'SOUTHMOOR PARK',
      'SPEER',
      'STAPLETON',
      'SUNNYSIDE',
      'SWANSEA',
      'UNIVERSITY',
      'UNIVERSITY ',
      'UNIVERSITY HILLS',
      'UNIVERSITY PARK',
      'VALVERDE',
      'VILLA PARK',
      'VILLAS CHRY HL',
      'VIRGINIA VILLAGE',
      'WASH PARK',
      'WELLSHIRE',
      'WESTBRIDGE',
      'WESTWOOD',
      'WHITTIER',
      'WILLOW POINT E',
      'WINDSOR',
      'WOODSTREAM',
      'ZUNI PARK']:

      model_path = os.path.join(model_dir, neighborhood + '.joblib')
      model_by_neighborhood[neighborhood] = joblib.load(model_path)

    return cls(model_by_neighborhood)   

Overwriting predictor.py


In [0]:
MODEL_NAME = 'DenverHousingPredictor2'
VERSION_NAME = 'v1'

In [0]:
%%writefile setup.py
from setuptools import setup

MODEL_NAME = 'DenverHousingPredictor2'

setup(
    name=MODEL_NAME,
    version='0.1',
    scripts=['predictor.py'])

Overwriting setup.py


In [0]:
! python setup.py sdist --formats=gztar

running sdist
running egg_info
creating DenverHousingPredictor2.egg-info
writing DenverHousingPredictor2.egg-info/PKG-INFO
writing dependency_links to DenverHousingPredictor2.egg-info/dependency_links.txt
writing top-level names to DenverHousingPredictor2.egg-info/top_level.txt
writing manifest file 'DenverHousingPredictor2.egg-info/SOURCES.txt'
reading manifest file 'DenverHousingPredictor2.egg-info/SOURCES.txt'
writing manifest file 'DenverHousingPredictor2.egg-info/SOURCES.txt'

running check


creating DenverHousingPredictor2-0.1
creating DenverHousingPredictor2-0.1/DenverHousingPredictor2.egg-info
copying files to DenverHousingPredictor2-0.1...
copying predictor.py -> DenverHousingPredictor2-0.1
copying setup.py -> DenverHousingPredictor2-0.1
copying DenverHousingPredictor2.egg-info/PKG-INFO -> DenverHousingPredictor2-0.1/DenverHousingPredictor2.egg-info
copying DenverHousingPredictor2.egg-info/SOURCES.txt -> DenverHousingPredictor2-0.1/DenverHousingPredictor2.egg-info
copying Den

In [0]:
! gsutil cp ./dist/$MODEL_NAME-0.1.tar.gz gs://$BUCKET_NAME/$MODEL_NAME/$MODEL_NAME-0.1.tar.gz

Copying file://./dist/DenverHousingPredictor2-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                                      


In [0]:
! gcloud ai-platform versions delete $VERSION_NAME --model $MODEL_NAME

This will delete version [v1]...

Do you want to continue (Y/n)?  Y



In [0]:
! gcloud ai-platform models delete $MODEL_NAME

This will delete model [DenverHousingPredictor]...

Do you want to continue (Y/n)?  Y



In [0]:
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

Created ml engine model [projects/denver-housing-project/models/DenverHousingPredictor2].


In [0]:
# --quiet automatically installs the beta component if it isn't already installed 
! gcloud --quiet beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://$BUCKET_NAME/model/ \
  --package-uris gs://$BUCKET_NAME/$MODEL_NAME/$MODEL_NAME-0.1.tar.gz \
  --prediction-class predictor.DenverHousingPredictor

In [0]:
! pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.7.10)


In [0]:
import googleapiclient.discovery


instances = [
    [square_feet, bedrooms, age_of_dwelling, full_bathrooms, land_size_sq_ft, condo, row_house, single_family]
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)
#project=denver-housing-project

response = service.projects().predict(
    name=name,
    body={'instances': instances, 'neighborhood': neighborhood }
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])


[[796471.6294528076]]


In [0]:
import os 
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib

model = joblib.load('./Stapleton.joblib')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
%%writefile predictor.py
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib

class MyPredictor(object):
  def __init__(self, model, preprocessor=None):
    self._model = model
#     self._preprocessor = preprocessor
    self._class_names = load_iris().target_names ## NATE REVIEW
#
  def predict(self, instances, **kwargs):
    inputs = np.asarray(instances) ## NATE REVIEW
    if kwargs.get('probabilities'):
      probabilities = self._model.predict_proba(inputs)
      return probabilities.tolist()
    else:
      outputs = self._model.predict(inputs)
      return [self._class_names[class_num] for class_num in outputs]

  @classmethod
  def from_path(cls, Models):
    neighborhood = 'Stapleton'
    model_path = os.path.join(Models, neighborhood +'.joblib') ## NATE REVIEW
    model = joblib.load(model_path)

#     preprocessor_path = os.path.join(Models, 'preprocessor.pkl') ## NATE REVIEW
#     with open(preprocessor_path, 'rb') as f:
#       preprocessor = pickle.load(f)

    return cls(model, None)

Overwriting predictor.py


In [0]:
%%writefile setup.py
from setuptools import setup

setup(
    name='my_custom_code',
    version='0.1',
    scripts=['predictor.py'])#, 'preprocess.py'])

Overwriting setup.py


In [0]:
! python setup.py sdist --formats=gztar

running sdist
running egg_info
creating my_custom_code.egg-info
writing my_custom_code.egg-info/PKG-INFO
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing top-level names to my_custom_code.egg-info/top_level.txt
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-0.1
creating my_custom_code-0.1/my_custom_code.egg-info
copying files to my_custom_code-0.1...
copying predictor.py -> my_custom_code-0.1
copying setup.py -> my_custom_code-0.1
copying my_custom_code.egg-info/PKG-INFO -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/SOURCES.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/dependency_links.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/top_level.txt -> my_custom_code-0.1/my_custom_code.

In [0]:
BUCKET_NAME = 'denver-neighborhood-model'
REGION = 'us-central1'

In [0]:

! gsutil cp ./dist/my_custom_code-0.1.tar.gz gs://$BUCKET_NAME/Models/my_custom_code-0.1.tar.gz ## NATE REVIEW
  
# ! gsutil cp model.joblib preprocessor.pkl gs://$BUCKET_NAME/Models ## NATE REVIEW

Copying file://./dist/my_custom_code-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


In [0]:
MODEL_NAME = 'SalesPredictor'
VERSION_NAME = 'v1'


In [0]:
! gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&access_type=offline


Enter verification code: 4/nQGyrmGkcfI6ZZHZ7icVEBgBTuRvkFE-DoAcUszhOL-hQxkI3fBqB98
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [jrmiller24@gmail.com].
Your current project is [denver-housing-project].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
! gcloud config set project denver-housing-project

Updated property [core/project].


In [0]:
print(MODEL_NAME)

SalesPredictor


Gotta remind meyself, what *is* the model name

In [0]:
! gcloud ai-platform versions delete v1 --model $MODEL_NAME
! gcloud ai-platform models delete $MODEL_NAME

This will delete version [v1]...

Do you want to continue (Y/n)?  Y

This will delete model [SalesPredictor]...

Do you want to continue (Y/n)?  Y



In [0]:
! gcloud ai-platform models create $MODEL_NAME \
  --regions us-central1

Created ml engine model [projects/denver-housing-project/models/SalesPredictor].


In [0]:
# --quiet automatically installs the beta component if it isn't already installed 
! gcloud --quiet beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://$BUCKET_NAME/Models \
  --package-uris gs://$BUCKET_NAME/Models/my_custom_code-0.1.tar.gz \
  --prediction-class predictor.MyPredictor

ERROR: (gcloud.beta.ai-platform.versions.create) FAILED_PRECONDITION: Field: package_uris Error: The provided GCS paths [gs://du-data-viz-bucket/Models/my_custom_code-0.1.tar.gz] cannot be read. Please make sure that the objects exist and you have read access to it.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The provided GCS paths [gs://du-data-viz-bucket/Models/my_custom_code-0.1.tar.gz]
      cannot be read. Please make sure that the objects exist and you have read access
      to it.
    field: package_uris


In [0]:
! pip install --upgrade google-api-python-client

In [0]:
import googleapiclient.discovery

instances = [
    square_feet, bedrooms, age_of_dwelling, full_bathrooms, land_size_sq_ft, condo, row_house, single_family
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

In [0]:
response = service.projects().predict(
    name=name,
    body={'instances': instances, 'probabilities': True}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

In [0]:
# Delete version resource
! gcloud ai-platform versions delete $VERSION_NAME --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$BUCKET_NAME/custom_prediction_routine_tutorial